In [0]:
from pyspark.sql.functions import * 

## uploaded manually to the unity catalog
fct_credit_risk = spark.table("workspace.default.fct_credit_risk")
mart_monthly_risk_summary = spark.table("workspace.default.mart_monthly_risk_summary")

In [0]:
df_geo_risk = (
    fct_credit_risk.groupBy("buyer_state")
    .agg(
        count("asset_id").alias("loan_count"),
        sum("face_value").alias("total_face_value"),
        mean("face_value").alias("avg_ticket_size"),
    )
    .orderBy(col("total_face_value").desc())
)

print("--- Top 5 States by Exposure ---")
df_geo_risk.show(5)

## PE accounts > ~60% of all loans and total face value
## portfolio is not nationally distributed, heavily skewed towards a single state
## any regulatory or economic shift specific to Pernambuco represents a systemic risk to this portfolio

--- Top 5 States by Exposure ---
+-----------+----------+--------------------+-----------------+
|buyer_state|loan_count|    total_face_value|  avg_ticket_size|
+-----------+----------+--------------------+-----------------+
|         PE|    213564| 1.606049215600021E8|752.0224455432661|
|         CE|     66153| 4.780487800999905E7|722.6411199794272|
|         SP|     24209|1.8604155300000325E7|768.4809492337695|
|         PB|     18640|1.4404782410000294E7|772.7887559013034|
|         MG|      6131|  4387710.9100000225|715.6599102919625|
+-----------+----------+--------------------+-----------------+
only showing top 5 rows


In [0]:
df_active_risk = fct_credit_risk.filter(
    col("collection_status").isin(["Active", "Defaulted"])
)

df_rating_analysis = (
    df_active_risk.groupBy("current_rating")
    .agg(
        round(mean("days_overdue"), 1).alias("avg_days_overdue"),
        round(mean("provision_rate"), 4).alias("avg_provision_rate"),
        count("asset_id").alias("loan_count"),
    )
    .orderBy("current_rating")
)

print("--- Risk Metrics by Rating (Active/Defaulted Only) ---")
df_rating_analysis.show()

## one would expect worse ratings to always have higher average days overdue
## but 'F' loans have an avg of 116 days overdue which is aproximate to 'A'
## worst performers are actually found in 'C' (173 days) and 'D' (168 days), not in the ostensibly "worst"
## 'F' might be a special classification rather than a linear progression of risk

--- Risk Metrics by Rating (Active/Defaulted Only) ---
+--------------+----------------+------------------+----------+
|current_rating|avg_days_overdue|avg_provision_rate|loan_count|
+--------------+----------------+------------------+----------+
|             A|           115.1|            0.7274|      2063|
|             B|           157.2|            0.8626|      2081|
|             C|           173.0|             0.896|     12270|
|             D|           168.6|            0.9015|      9000|
|             E|           149.0|             0.884|      2480|
|             F|           116.3|            0.8357|       409|
|             G|           133.5|            0.8148|        27|
+--------------+----------------+------------------+----------+



In [0]:
df_trend = (
    mart_monthly_risk_summary.groupBy("risk_month")
    .agg(
        sum("total_exposure").alias("total_exposure"),
        sum("total_expected_loss").alias("total_expected_loss"),
    )
    .withColumn("loss_ratio", col("total_expected_loss") / col("total_exposure"))
    .orderBy("risk_month")
)

df_trend.orderBy(col("risk_month").desc()).show()

## for active/defaulted loans, the lowest average provision rate is 72% for A
## scales up to ~90% for C/D
## business takes a very conservative stance, provisioning nearly the full value of the loan once it becomes problematic (regardless of rating)

+----------+--------------------+-------------------+--------------------+
|risk_month|      total_exposure|total_expected_loss|          loss_ratio|
+----------+--------------------+-------------------+--------------------+
|2026-03-01|           383132.24| 240.70600000000002|6.282582744798507E-4|
|2026-02-01|  2818335.1199999996|         51402.3578| 0.01823855418584856|
|2026-01-01|   7441130.669999997|         76429.5635|0.010271229855986393|
|2025-12-01|2.1250421039999988E7| 203977.68950000004| 0.00959875990767664|
|2025-11-01|2.6464821350000005E7| 328296.57690000004| 0.01240501768586471|
|2025-10-01|2.9572356730000008E7|         2256089.97| 0.07629050300584547|
|2025-09-01| 2.317829382999998E7| 1885811.2099999997| 0.08136108825918709|
|2025-08-01|2.0661168079999994E7|         1531213.58| 0.07411069761744082|
|2025-07-01|1.9187428430000003E7|          1930388.6| 0.10060694725416103|
|2025-06-01|       1.758043088E7| 1382418.2400000002| 0.07863392253785308|
|2025-05-01|1.58400544499

In [0]:
df_toxic_segments = (
    mart_monthly_risk_summary.groupBy("segment")
    .agg(
        sum("total_exposure").alias("total_exposure"),
        sum("total_expected_loss").alias("total_expected_loss"),
    )
    .withColumn("loss_ratio", col("total_expected_loss") / col("total_exposure"))
    .orderBy(col("loss_ratio").desc())
)

df_toxic_segments.show()

## Pernambuco dominates the volume and performs reasonably well with a ~6.8% loss ratio
## Bahia has massive 33.6% loss ratio as well as Tocantins (21.3%).
## risk models might be over-fitting for the major markets and/or fail to capture local risks in specific frontier states

+-------+--------------------+--------------------+--------------------+
|segment|      total_exposure| total_expected_loss|          loss_ratio|
+-------+--------------------+--------------------+--------------------+
|     BA|  214040.83999999997|   72049.79699999999| 0.33661705401642045|
|     TO|             16521.1|            3519.339| 0.21302086422816885|
|     RR|  30566.359999999997|             5609.16| 0.18350762079619556|
|     PR|   287520.4700000001|  33881.605899999995| 0.11784067374402937|
|     RO|  3786.1900000000005|              430.81| 0.11378456971256064|
|     AM|  102049.35999999999|           8120.0175|   0.079569509304125|
|     PB|1.4404491899999995E7|  1125235.5562999998| 0.07811699045767802|
|     PE|1.6038445436999995E8|1.0984506606800001E7| 0.06848859915973667|
|     SP|       1.860393096E7|        1270626.7904| 0.06829883389332896|
|     MA|   86728.37000000001|   5756.471599999999| 0.06637357072432006|
|     CE| 4.780487801000002E7|  3048152.7841000003|

In [0]:
df_vintage = (
    fct_credit_risk.withColumn("vintage_month", date_format("created_at", "yyyy-MM"))
    .groupBy("vintage_month")
    .agg(
        count("asset_id").alias("total_loans"),
        sum(when(col("collection_status") == "Defaulted", 1).otherwise(0)).alias(
            "defaults"
        ),
    )
    .withColumn("default_rate", col("defaults") / col("total_loans"))
    .orderBy("vintage_month")
)

df_vintage.show()

## extremely short portfolio (14 to 24 days)
## correlation where better-rated loans get longer terms (A ~22 days) while riskier loans (Ratings C & D) are kept on a tighter leash (~14 days)
## business has risk mitigation tool—shortening exposure time for riskier borrowers

+-------------+-----------+--------+--------------------+
|vintage_month|total_loans|defaults|        default_rate|
+-------------+-----------+--------+--------------------+
|      2024-01|          3|       0|                 0.0|
|      2024-02|         11|       1| 0.09090909090909091|
|      2024-03|          8|       0|                 0.0|
|      2024-04|        300|      18|                0.06|
|      2024-05|        404|      32| 0.07920792079207921|
|      2024-06|        294|      18|0.061224489795918366|
|      2024-07|       1258|     109| 0.08664546899841018|
|      2024-08|       2641|     210| 0.07951533510034078|
|      2024-09|       4809|     395| 0.08213765855687254|
|      2024-10|       7741|     701| 0.09055677561038626|
|      2024-11|       8726|     766| 0.08778363511345405|
|      2024-12|      14910|    1265| 0.08484238765928907|
|      2025-01|      16598|    1418| 0.08543197975659718|
|      2025-02|      15644|    1348| 0.08616722065967783|
|      2025-03

In [0]:
df_whales = (
    fct_credit_risk.groupBy("buyer_tax_id")
    .agg(
        sum("face_value").alias("total_exposure"),
        mean("days_overdue").alias("avg_overdue"),
        max("days_overdue").alias("max_overdue"),
    )
    .orderBy(col("total_exposure").desc())
)

df_whales.show(10)

# top 10 borrowers hold ~8.1% of the entire portfolio value

+--------------+------------------+------------------+-----------+
|  buyer_tax_id|    total_exposure|       avg_overdue|max_overdue|
+--------------+------------------+------------------+-----------+
|31591170337004|3145036.6000000006| 72.38461538461539|        221|
|39951072246362|        2841662.83| 69.70769230769231|        249|
|60940116617141|2557980.2199999997| 126.9047619047619|        246|
|94120420875235|2014176.6299999997|38.719298245614034|        305|
|70757826524746|1974879.9900000007|35.303030303030305|        272|
|81293483643299|        1955536.34|44.422222222222224|        272|
|52092831896096|         1541684.3|             172.2|        272|
|72674612228519|1493723.8399999999| 35.46153846153846|        141|
|50477193611057|1396078.2000000002|             108.2|        216|
|16987395891041|        1341627.64|138.88235294117646|        264|
+--------------+------------------+------------------+-----------+
only showing top 10 rows


In [0]:
df_ticket_risk = (
    fct_credit_risk.withColumn(
        "ticket_bin",
        when(col("face_value") <= 100, "0-100")
        .when(col("face_value") <= 500, "100-500")
        .when(col("face_value") <= 1000, "500-1k")
        .when(col("face_value") <= 10000, "1k-10k")
        .otherwise("10k+"),
    )
    .groupBy("ticket_bin")
    .agg(
        count("asset_id").alias("count"),
        mean(when(col("collection_status") == "Defaulted", 1.0).otherwise(0.0)).alias(
            "default_rate"
        ),
    )
    .orderBy("ticket_bin")
)

df_ticket_risk.show()

## 100-500 loans have a 7.5% default rate compared to 5.4% for loans >10k
## while whales (large loans) are dangerous if one fails, the "death by a thousand cuts" comes from the mid-to-small segment
## credit scoring model seems to work better for larger loans than for smaller

+----------+------+-------------------+
|ticket_bin| count|       default_rate|
+----------+------+-------------------+
|     0-100| 66367|0.07306342007322916|
|   100-500|176538|0.07504899794944998|
|      10k+|  2287|0.05421950153038916|
|    1k-10k| 39651|0.06748884013013544|
|    500-1k| 50169|0.06994359066355718|
+----------+------+-------------------+



In [0]:
top_states = ["PE", "CE", "SP", "PB", "MG"]

df_state_rating = (
    fct_credit_risk.filter(col("buyer_state").isin(top_states))
    .groupBy("buyer_state", "current_rating")
    .count()
)

df_pivot = df_state_rating.groupBy("buyer_state").pivot("current_rating").sum("count")

df_pivot.show()

## all top 5 states have approximate rating distributions
## massive concentration in PE is not because the algorithm rates people there better
## could be commercial/sales driver

+-----------+-----+-----+-----+-----+-----+----+---+
|buyer_state|    A|    B|    C|    D|    E|   F|  G|
+-----------+-----+-----+-----+-----+-----+----+---+
|         CE| 4866| 4852|28671|20961| 5746|1006| 51|
|         PE|15186|15820|92264|67825|19027|3267|175|
|         PB| 1309| 1291| 8053| 5986| 1674| 315| 12|
|         MG|  415|  480| 2682| 1916|  543|  87|  8|
|         SP| 1781| 1773|10500| 7635| 2112| 391| 17|
+-----------+-----+-----+-----+-----+-----+----+---+



In [0]:
## despite rating anomalies found earlier, aggregate portfolio performance is stable
## for mature vintages (Feb/2025 – Jul/2025) default rate consistently hovers between 8.4% and 8.6%
## stability suggests that macro-level risk controls are working, creating a predictable "cost of doing business" around 8.5%